# Import libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
import pandas as pd  
import numpy as np
from statsmodels import robust
from pandas import read_excel
from scipy import spatial
from numpy.linalg import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import mplcursors
import random
from matplotlib import gridspec
import tensorflow as tf
import os

# 1. Read the data


In [3]:
file_name = "/home/gauss//Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized/alldata.xlsx"
data = pd.read_excel(file_name)
data['Plate number'] = data['Plate number'].apply(str)
data.head()

,Unnamed: 0,Plate number,Combination,Unnamed: 0.1,feature1,feature2,feature3,feature4,feature5,feature6,...,feature509,feature510,feature511,feature512,Well_number,Compound,MoA,Concentration,Label,Category
0,0,1,O8W,0,-3.250606,-0.558769,1.742629,-0.137684,-5.287768,1.228142,...,3.424731,0.977335,-1.322238,-7.194132,A1,cells,cells,None,cells,Negative Controls
1,1,1,O8W,1,8.104449,-41.294835,-21.201680,25.177647,-91.764581,-28.949970,...,41.672861,9.687123,58.462101,70.505096,A2,BzCl,BzCl,0,BzCl,Miscl.
2,2,1,O8W,2,53.977080,-50.731853,-4.885258,18.007128,-60.000932,-30.644327,...,42.691672,22.679383,38.773556,51.500469,A3,Vorinostat,HDAC inhibitor,10000,Drug,HDAC
3,3,1,O8W,3,-3.251336,-2.373047,-1.217898,0.777455,-2.873282,-0.734402,...,3.575650,-0.487380,-0.026772,-5.626232,A4,Bicalutamide,Nonsteriodal antiandrogen,10000,Drug,Miscl.
4,4,1,O8W,4,0.245413,0.489912,1.108443,-0.082875,-4.435874,-0.559726,...,0.075624,-0.035285,1.948833,5.054140,A5,Prednisolone,"Glucocorticoid, immunomodulatory agent",10000,Drug,Miscl.


# 2. Correlation matrix 

## To calculate enrichment score there are two ways:
### 1. Compute correlation matrix across selected MoA and then calculate Enrichment

In [4]:
#cat = ['EGFR', 'VEGFR', 'PI3K', 'Topoisomerase', 'Mitotic', 'MEK1/2', 'PARP', 'CDK', 'BET', 'HDAC']
#data = data.loc[data['Category'].isin(cat)]
#data

#### 1.1 Calculate enrichment for each combination i.e KB, O3B, MHB etc 

In [5]:
#if correlation matrix across entire 8 plates for each combination is required
cat = ["O8W"]
data = data.loc[data['Combination'].isin(cat)]
data


,Unnamed: 0,Plate number,Combination,Unnamed: 0.1,feature1,feature2,feature3,feature4,feature5,feature6,...,feature509,feature510,feature511,feature512,Well_number,Compound,MoA,Concentration,Label,Category
0,0,1,O8W,0,-3.250606,-0.558769,1.742629,-0.137684,-5.287768,1.228142,...,3.424731,0.977335,-1.322238,-7.194132,A1,cells,cells,None,cells,Negative Controls
1,1,1,O8W,1,8.104449,-41.294835,-21.201680,25.177647,-91.764581,-28.949970,...,41.672861,9.687123,58.462101,70.505096,A2,BzCl,BzCl,0,BzCl,Miscl.
2,2,1,O8W,2,53.977080,-50.731853,-4.885258,18.007128,-60.000932,-30.644327,...,42.691672,22.679383,38.773556,51.500469,A3,Vorinostat,HDAC inhibitor,10000,Drug,HDAC
3,3,1,O8W,3,-3.251336,-2.373047,-1.217898,0.777455,-2.873282,-0.734402,...,3.575650,-0.487380,-0.026772,-5.626232,A4,Bicalutamide,Nonsteriodal antiandrogen,10000,Drug,Miscl.
4,4,1,O8W,4,0.245413,0.489912,1.108443,-0.082875,-4.435874,-0.559726,...,0.075624,-0.035285,1.948833,5.054140,A5,Prednisolone,"Glucocorticoid, immunomodulatory agent",10000,Drug,Miscl.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,379,8,O8W,379,-2.338976,0.188733,1.259004,-0.860685,-1.515621,0.676822,...,-2.037654,-0.204069,0.045548,-3.456309,P20,BRD7116,Leukemic stem cell inhibitor,10000,Drug,Miscl.
3068,380,8,O8W,380,-3.251973,1.815991,-0.867979,0.582528,0.147987,1.951520,...,0.858306,-2.174467,0.659268,-2.588171,P21,JPH203,LAT1 inhibitor,5000,Drug,Miscl.
3069,381,8,O8W,381,-1.144508,1.056336,1.391212,0.560384,-2.814150,0.785845,...,-0.473392,0.655326,0.271642,-1.269701,P22,DMSO,DMSO,0,DMSO,Negative Controls
3070,382,8,O8W,382,-0.342106,-1.464002,1.141274,2.952085,-1.963907,0.314911,...,0.013665,0.817295,0.584412,-0.424555,P23,RSL3,"GPX4 inhibitor, induces ferroptosis",10000,Drug,Miscl.


### 2. Compute correlation matrix across entire dataset and then compute Enrichment

In [6]:
new_data = data.drop(columns=[ "Unnamed: 0","Unnamed: 0.1", "Concentration", "Label"], axis=1) #"Unnamed: 0.1.1"

new_data['well_annotation'] =  new_data['Plate number']+ '-' + new_data['Combination']+ '-' + new_data['Well_number'] + '-' + new_data['Compound'] +  '-' + new_data['MoA'] 

new_data['well_num'] =  new_data['Plate number'] + '-' + new_data['Combination']+ '-' + new_data['Well_number']


data_new = new_data.drop(columns=["Plate number", "Combination","Compound","Well_number", "MoA", "Category", "well_annotation", "well_num"], axis=1)

#prepare dataframe for correlation matrix
data_corr = new_data.drop(columns=["Plate number", "Combination", "Compound", "MoA", "Category", "well_annotation", "Well_number"], axis=1)
#Put well_num col as first and set it as index col
first_column = data_corr.pop('well_num')
data_corr.insert(0, 'well_num', first_column)
data_corr.set_index('well_num', inplace=True)

#transpose the dataframe and compute coorelation coefficient
data_new_T = data_corr.T
correlation_mat = data_new_T.corr()
correlation_mat.head()

well_num,1-O8W-A1,1-O8W-A2,1-O8W-A3,1-O8W-A4,1-O8W-A5,1-O8W-A6,1-O8W-A7,1-O8W-A8,1-O8W-A9,1-O8W-A10,...,8-O8W-P15,8-O8W-P16,8-O8W-P17,8-O8W-P18,8-O8W-P19,8-O8W-P20,8-O8W-P21,8-O8W-P22,8-O8W-P23,8-O8W-P24
well_num,,,,,,,,,,,,,,,,,,,,,
1-O8W-A1,1.000000,-0.048367,0.058212,0.519481,0.351091,0.290367,-0.011200,0.035306,0.161020,0.092461,...,-0.006897,-0.042440,-0.025242,-0.005557,0.040970,0.062057,0.205368,0.022072,0.082326,-0.039314
1-O8W-A2,-0.048367,1.000000,0.823288,-0.081325,0.281739,-0.090354,0.894493,0.865091,-0.008665,-0.092712,...,0.882350,-0.273165,0.867875,0.874531,0.053859,-0.216399,0.068035,-0.054213,0.242429,0.894868
1-O8W-A3,0.058212,0.823288,1.000000,-0.032067,0.313066,-0.104964,0.790513,0.838044,0.018833,-0.110581,...,0.821284,-0.285146,0.767953,0.832234,0.008816,-0.274696,0.081587,-0.097316,0.261814,0.808402
1-O8W-A4,0.519481,-0.081325,-0.032067,1.000000,-0.052388,0.362967,0.005704,-0.060582,0.233167,0.237548,...,-0.026677,0.020694,-0.043672,-0.044765,0.096700,-0.097490,0.218390,-0.079417,0.201020,-0.039627
1-O8W-A5,0.351091,0.281739,0.313066,-0.052388,1.000000,-0.064492,0.282421,0.386239,-0.015646,-0.172808,...,0.287946,-0.306277,0.259089,0.261149,-0.023077,-0.282345,-0.112260,0.173951,0.387606,0.251686


In [7]:
data_corr

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature503,feature504,feature505,feature506,feature507,feature508,feature509,feature510,feature511,feature512
well_num,,,,,,,,,,,,,,,,,,,,,
1-O8W-A1,-3.250606,-0.558769,1.742629,-0.137684,-5.287768,1.228142,-2.313747,-1.845029,-0.368448,-0.764416,...,0.451769,1.342948,1.980261,1.605276,-2.897184,-1.156124,3.424731,0.977335,-1.322238,-7.194132
1-O8W-A2,8.104449,-41.294835,-21.201680,25.177647,-91.764581,-28.949970,6.764639,56.376224,3.472097,-24.825987,...,24.864598,9.257125,-0.488050,-25.659255,-20.411542,46.673237,41.672861,9.687123,58.462101,70.505096
1-O8W-A3,53.977080,-50.731853,-4.885258,18.007128,-60.000932,-30.644327,2.042808,63.898988,5.455327,-17.188768,...,52.758058,-4.250835,0.576638,-26.585208,11.854736,58.278037,42.691672,22.679383,38.773556,51.500469
1-O8W-A4,-3.251336,-2.373047,-1.217898,0.777455,-2.873282,-0.734402,-1.041985,-1.792717,-0.252898,-1.318357,...,2.809749,2.149136,1.567790,0.693152,-0.726504,-1.586127,3.575650,-0.487380,-0.026772,-5.626232
1-O8W-A5,0.245413,0.489912,1.108443,-0.082875,-4.435874,-0.559726,-0.298827,1.270686,1.637415,-2.341989,...,-0.177615,-0.711005,0.085576,0.585991,-3.650917,1.657142,0.075624,-0.035285,1.948833,5.054140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8-O8W-P20,-2.338976,0.188733,1.259004,-0.860685,-1.515621,0.676822,-2.679147,-2.026892,-2.855320,4.022502,...,-5.847153,0.985123,1.636104,1.793029,-0.224317,0.349174,-2.037654,-0.204069,0.045548,-3.456309
8-O8W-P21,-3.251973,1.815991,-0.867979,0.582528,0.147987,1.951520,-4.105907,0.003569,-3.125665,1.207625,...,-1.750613,0.621749,2.717510,2.027478,-2.222332,0.094017,0.858306,-2.174467,0.659268,-2.588171
8-O8W-P22,-1.144508,1.056336,1.391212,0.560384,-2.814150,0.785845,1.070783,-0.264614,0.002124,-0.012855,...,-3.896725,0.270876,-0.522930,0.165332,-0.005761,0.210580,-0.473392,0.655326,0.271642,-1.269701


In [8]:
X = data_new.iloc[:,:].values #features on which distance will be calculated
y = new_data["well_annotation"] #well annotation to map the distance index later
z = new_data["Category"]
conc = data["Concentration"] #change to "data" in case using full dataframe
y = y.values
z = z.values
conc = conc.values
print(X)
print(y)
print(z)
print(conc)

[[ -3.25060565  -0.55876913   1.7426294  ...   0.97733484  -1.32223768
   -7.19413244]
 [  8.1044494  -41.29483465 -21.20167992 ...   9.68712256  58.46210067
   70.50509551]
 [ 53.97708046 -50.73185283  -4.88525756 ...  22.67938307  38.7735564
   51.50046911]
 ...
 [ -1.14450848   1.05633608   1.39121249 ...   0.65532612   0.27164218
   -1.26970075]
 [ -0.34210576  -1.46400196   1.14127382 ...   0.8172946    0.5844117
   -0.42455477]
 [  4.83699461 -36.75726249  -7.52366219 ...  28.98566483  58.49328544
   26.12476683]]
['1-O8W-A1-cells-cells' '1-O8W-A2-BzCl-BzCl'
 '1-O8W-A3-Vorinostat-HDAC inhibitor' ... '8-O8W-P22-DMSO-DMSO'
 '8-O8W-P23-RSL3-GPX4 inhibitor, induces ferroptosis'
 '8-O8W-P24-BzCl-BzCl']
['Negative Controls' 'Miscl.' 'HDAC' ... 'Negative Controls' 'Miscl.'
 'Miscl.']
['None' '0' '10000' ... '0' '10000' '0']


# 4. Calculating enrichment score and Pvalue

In [ ]:
well_annot = []
well_conc = []
well_cat = []
highest_es = []
pval = []

moa = ['EGFR', 'VEGFR', 'PI3K', 'Topoisomerase', 'Mitotic', 'MEK1/2', 'PARP', 'CDK', 'BET', 'HDAC']

for m in range(len(moa)):
    print(moa[m])
    categories = new_data.loc[new_data['Category'] == moa[m]].index.tolist()
    cat_moa = moa[m]
    
    for i in range(len(categories)):
        print(i)
        # Create a dataframe with well annotations, category, concentration, and correlation coefficient with respect to that well
        data_enrch = pd.DataFrame({'Well_annotation': y,
                                   'Well_category': z,
                                   'Well_concentration': conc,
                                   'Correlation_matrix': correlation_mat[new_data['well_num'][categories[i]]].tolist()})
        data_enrch = data_enrch.sort_values(by='Correlation_matrix', ascending=False).reset_index()


        data_enrch['res'] = 'nan'
        

        selected_moa = data_enrch.loc[data_enrch['Well_category'] == cat_moa]
        M = (selected_moa['Correlation_matrix'].abs()).sum()
        
        T = (data_enrch['Well_category']!= cat_moa).sum()
        
        
        data_enrch.at[0, 'res'] = data_enrch.at[0, 'Correlation_matrix'] * (1/M)
        
        
        for j in range(1, len(data_enrch)): 
        
            if data_enrch.at[j, 'Well_category'] == cat_moa:
                data_enrch.at[j, 'res'] = data_enrch.at[j-1, 'res'] + (data_enrch.at[j, 'Correlation_matrix']* (1 / M))
                
            elif data_enrch.at[j, 'Well_category'] != cat_moa:
                data_enrch.at[j, 'res'] = data_enrch.at[j-1, 'res'] - (1 / T)
                
        res_original = max(data_enrch['res']) 
        

        res_max = []

        #Random shuffle of MoA Category labels 
        for i in range(0,1000):
            first_well_category = data_enrch.loc[0, 'Well_category']
            well_category  = data_enrch['Well_category'].tolist()
            rest_well_category = well_category[1:]
            random.shuffle(rest_well_category)
            Well_category_col = [first_well_category] + rest_well_category
            data_enrch['Well_category'] = Well_category_col
            
        #Compute M and T for shuffled labels data
            selected_moa = data_enrch.loc[data_enrch['Well_category'] == cat_moa]
            M = (selected_moa['Correlation_matrix'].abs()).sum()
            T = (data_enrch['Well_category']!= cat_moa).sum()
            
        #Compute new Res based on shuffled labels 
            data_enrch.at[0, 'res'] = data_enrch.at[0, 'Correlation_matrix'] * (1/M)
            for j in range(1, len(data_enrch)): 
                if data_enrch.at[j, 'Well_category'] == cat_moa:
                    data_enrch.at[j, 'res'] = data_enrch.at[j-1, 'res'] + (data_enrch.at[j, 'Correlation_matrix']* (1 / M))
                elif data_enrch.at[j, 'Well_category'] != cat_moa:
                    data_enrch.at[j, 'res'] = data_enrch.at[j-1, 'res'] - (1 / T)
            #save the highest res from eaxh shuffle 
            res_max.append(max(data_enrch['res']))
        
        #Compute Pval comparing highest Res from each shuffle to the original highest Res without shuffle data
        c = 0
        for ele in res_max:
            if ele >= res_original:
                c = c+1
        p_val = c/1000

        
        well_annot.append(data_enrch['Well_annotation'][0])
        well_conc.append(data_enrch['Well_concentration'][0])
        well_cat.append(cat_moa)
        highest_es.append(res_original)
        pval.append(p_val)


        data_Pval = pd.DataFrame({'Well_annotation': well_annot, 'Well_concentration': well_conc, 'Well_Category': well_cat, 'Highest_ES': highest_es, 'Pvalue': pval})


EGFR
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
VEGFR
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
PI3K
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
Topoisomerase
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21

In [ ]:
data_Pval_enriched = data_Pval[data_Pval['Pvalue'] < 0.05]
data_Pval_enriched = data_Pval_enriched['Well_Category'].value_counts().rename_axis('MoA').reset_index(name='value')


In [ ]:
data_Pval_enriched

In [ ]:
data_Pval_enriched.to_excel("/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized/pval_norm_O8W.xlsx")
